In [ ]:
import pandas as pd
import ccxt
from tqdm import tqdm
import time


In [92]:
binance= ccxt.binance()


pairs=[pair for pair in binance.load_markets() if pair.endswith('ETH')]

print(len(pairs))


228


In [112]:
timeframes = ['5m','15m','30m','1h', '2h', '4h', '12h', '1d']
all_data = []

# Calculate the start date based on the number of candles we want
for timeframe in timeframes:
    for pair in tqdm(pairs):
        try:
            # Fetch last 10 candles directly
            candles = binance.fetch_ohlcv(pair, timeframe, limit=10)
            
            for candle in candles:
                timestamp, open_, high, low, close, volume = candle
                all_data.append({
                    "pair": pair,
                    "timeframe": timeframe,
                    "timestamp": pd.to_datetime(timestamp, unit="ms"),
                    "open": open_,
                    "high": high,
                    "low": low,
                    "close": close,
                    "volume": volume
                })
            
            time.sleep(1)  # Rate limiting
            
        except Exception as e:
            print(f"Error fetching data for {pair} on {timeframe}: {e}")

# Convert to DataFrame with timeframe information
df = pd.DataFrame(all_data)

# You can access specific timeframe data like this:
df_1h = df[df['timeframe'] == '1h']
df_2h = df[df['timeframe'] == '2h']
df_4h = df[df['timeframe'] == '4h']
df_12h = df[df['timeframe'] == '12h']
df_1d = df[df['timeframe'] == '1d']


  0%|          | 0/59 [00:00<?, ?it/s]

100%|██████████| 59/59 [01:16<00:00,  1.30s/it]


In [114]:
len(df)

4720

In [115]:
df_QTUM=df_1h[df_1h['pair']=='QTUM/ETH']
df_QTUM

,pair,timeframe,timestamp,open,high,low,close,volume
1770,QTUM/ETH,1h,2024-11-24 02:00:00,0.001090,0.001101,0.001088,0.001101,1400.8
1771,QTUM/ETH,1h,2024-11-24 03:00:00,0.001110,0.001140,0.001110,0.001140,3557.6
1772,QTUM/ETH,1h,2024-11-24 04:00:00,0.001135,0.001135,0.001123,0.001123,785.1
1773,QTUM/ETH,1h,2024-11-24 05:00:00,0.001131,0.001131,0.001117,0.001128,295.8
1774,QTUM/ETH,1h,2024-11-24 06:00:00,0.001133,0.001162,0.001133,0.001142,2711.6
1775,QTUM/ETH,1h,2024-11-24 07:00:00,0.001146,0.001146,0.001142,0.001142,356.0
1776,QTUM/ETH,1h,2024-11-24 08:00:00,0.001146,0.001148,0.001123,0.001123,820.3
1777,QTUM/ETH,1h,2024-11-24 09:00:00,0.001123,0.001138,0.001118,0.001138,882.2
1778,QTUM/ETH,1h,2024-11-24 10:00:00,0.001125,0.001125,0.001106,0.001106,914.9
1779,QTUM/ETH,1h,2024-11-24 11:00:00,0.001106,0.001106,0.001090,0.001090,479.3


In [117]:
pairs = df['pair'].unique()

for pair in pairs:
    for timeframe in timeframes:
        df_pair_timeframe = df[(df['pair'] == pair) & (df['timeframe'] == timeframe)].copy()
        df_pair_timeframe.loc[:, 'profit_percentage'] = (df_pair_timeframe['close'] - df_pair_timeframe['open'])*100/df_pair_timeframe['open']
        max_profit_index = df_pair_timeframe['profit_percentage'].idxmax()
        next_candels = df_pair_timeframe.loc[max_profit_index+1:]
        max_profit_price = df_pair_timeframe.loc[max_profit_index, 'close']
        percentage_changes = [(price / max_profit_price - 1) * 100 for price in next_candels['close']]
        
        has_positive = any(change > 0 for change in percentage_changes)
        has_fifty_negative = any(change <= -50 for change in percentage_changes)
        has_five_negative = any(change <= -5 for change in percentage_changes)

        if (not has_positive and not has_fifty_negative and has_five_negative):
            print(f"Pair: {pair} | Timeframe: {timeframe}")
        


Pair: SNT/ETH | Timeframe: 1h
Pair: SNT/ETH | Timeframe: 2h
Pair: SNT/ETH | Timeframe: 4h
Pair: SNT/ETH | Timeframe: 1d
Pair: BNB/ETH | Timeframe: 1d
Pair: LRC/ETH | Timeframe: 15m
Pair: LRC/ETH | Timeframe: 30m
Pair: LRC/ETH | Timeframe: 1h
Pair: LRC/ETH | Timeframe: 2h
Pair: IOTA/ETH | Timeframe: 12h
Pair: XVG/ETH | Timeframe: 4h
Pair: XVG/ETH | Timeframe: 1d
Pair: DASH/ETH | Timeframe: 30m
Pair: XRP/ETH | Timeframe: 4h
Pair: ENJ/ETH | Timeframe: 5m
Pair: ENJ/ETH | Timeframe: 15m
Pair: ENJ/ETH | Timeframe: 2h
Pair: XLM/ETH | Timeframe: 2h
Pair: LTC/ETH | Timeframe: 12h
Pair: LTC/ETH | Timeframe: 1d
Pair: ELF/ETH | Timeframe: 12h
Pair: ZEN/ETH | Timeframe: 30m
Pair: ZEN/ETH | Timeframe: 1h
Pair: ZEN/ETH | Timeframe: 4h
Pair: THETA/ETH | Timeframe: 4h
Pair: IOTX/ETH | Timeframe: 15m
Pair: IOTX/ETH | Timeframe: 30m
Pair: IOTX/ETH | Timeframe: 1h
Pair: IOTX/ETH | Timeframe: 4h
Pair: SC/ETH | Timeframe: 1d
Pair: VET/ETH | Timeframe: 4h
Pair: SLP/ETH | Timeframe: 1h
Pair: SLP/ETH | Timefra

In [ ]:
df.groupby('pair').count()

,timestamp,open,high,low,close,volume
pair,,,,,,
AAVE/ETH,10,10,10,10,10,10
ADA/ETH,10,10,10,10,10,10
ADX/ETH,10,10,10,10,10,10
APT/ETH,10,10,10,10,10,10
ARB/ETH,10,10,10,10,10,10
ATOM/ETH,10,10,10,10,10,10
AVAX/ETH,10,10,10,10,10,10
AXS/ETH,10,10,10,10,10,10
BNB/ETH,10,10,10,10,10,10


In [ ]:
df_TRX_ETH=df[df['pair']=='TRX/ETH']
df_TRX_ETH

,pair,timestamp,open,high,low,close,volume
120,TRX/ETH,2024-11-14,0.000056,0.000058,0.000055,0.000058,9885467.0
121,TRX/ETH,2024-11-15,0.000058,0.000063,0.000057,0.000062,8013731.0
122,TRX/ETH,2024-11-16,0.000062,0.000065,0.000060,0.000064,5978421.0
123,TRX/ETH,2024-11-17,0.000064,0.000065,0.000062,0.000065,5388965.0
124,TRX/ETH,2024-11-18,0.000065,0.000066,0.000063,0.000063,6395366.0
125,TRX/ETH,2024-11-19,0.000063,0.000066,0.000063,0.000064,4839706.0
126,TRX/ETH,2024-11-20,0.000064,0.000064,0.000063,0.000063,5803406.0
127,TRX/ETH,2024-11-21,0.000063,0.000064,0.000059,0.000059,6251356.0
128,TRX/ETH,2024-11-22,0.000059,0.000062,0.000059,0.000062,5717189.0
129,TRX/ETH,2024-11-23,0.000062,0.000065,0.000061,0.000064,5065783.0


In [ ]:
highest_candel=df_TRX_ETH[df_TRX_ETH['high']==df_TRX_ETH['high'].max()]
df_TRX_ETH['profit_percentage']=(df_TRX_ETH['close']-df_TRX_ETH['open'])*100/df_TRX_ETH['open']
df_TRX_ETH

/tmp/ipykernel_109395/2802328695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TRX_ETH['profit_percentage']=(df_TRX_ETH['close']-df_TRX_ETH['open'])*100/df_TRX_ETH['open']


,pair,timestamp,open,high,low,close,volume,profit_percentage
120,TRX/ETH,2024-11-14,0.000056,0.000058,0.000055,0.000058,9885467.0,3.924392
121,TRX/ETH,2024-11-15,0.000058,0.000063,0.000057,0.000062,8013731.0,7.141619
122,TRX/ETH,2024-11-16,0.000062,0.000065,0.000060,0.000064,5978421.0,3.249798
123,TRX/ETH,2024-11-17,0.000064,0.000065,0.000062,0.000065,5388965.0,1.407570
124,TRX/ETH,2024-11-18,0.000065,0.000066,0.000063,0.000063,6395366.0,-2.668518
125,TRX/ETH,2024-11-19,0.000063,0.000066,0.000063,0.000064,4839706.0,2.149339
126,TRX/ETH,2024-11-20,0.000064,0.000064,0.000063,0.000063,5803406.0,-1.291427
127,TRX/ETH,2024-11-21,0.000063,0.000064,0.000059,0.000059,6251356.0,-6.715006
128,TRX/ETH,2024-11-22,0.000059,0.000062,0.000059,0.000062,5717189.0,3.884479
129,TRX/ETH,2024-11-23,0.000062,0.000065,0.000061,0.000064,5065783.0,4.256010


In [ ]:
max_profit_index=df_TRX_ETH['profit_percentage'].idxmax()
max_profit_index

np.int64(121)

In [ ]:
next_candels=df_TRX_ETH.loc[max_profit_index+1:]
len(next_candels)

8

In [ ]:
# Get the max profit closing price first
max_profit_price = df_TRX_ETH.loc[max_profit_index, 'close']
print(f"Max Profit Price: {max_profit_price :.8f}")

# Calculate percentage changes relative to max profit candle
percentage_changes = [(price / max_profit_price - 1) * 100 for price in next_candels['close']]
percentage_changes


Max Profit Price: 0.00006181


[np.float64(3.31661543439572),
 np.float64(4.902119398155613),
 np.float64(2.087040931888029),
 np.float64(3.8019737906487494),
 np.float64(2.637113735641483),
 np.float64(-4.2549749231516),
 np.float64(-0.4853583562530406),
 np.float64(3.8343310143989617)]

In [ ]:
# Check conditions using percentage_changes
has_positive = any(change > 0 for change in percentage_changes)
has_fifty_negative = any(change <= -50 for change in percentage_changes)
has_five_negative = any(change <= -5 for change in percentage_changes)

# Print pair and timeframe if conditions are met
if (not has_positive and not has_fifty_negative and has_five_negative):
    print(f"Pair: TRX/ETH")
    print(f"Timeframe: {timeframe}")


In [ ]:
pairs = df['pair'].unique()
for pair in pairs:
    df_pair_timeframe = df[df['pair']==pair].copy()  # Create an explicit copy
    df_pair_timeframe.loc[:, 'profit_percentage'] = (df_pair_timeframe['close'] - df_pair_timeframe['open'])*100/df_pair_timeframe['open']
    max_profit_index = df_pair_timeframe['profit_percentage'].idxmax()
    next_candels = df_pair_timeframe.loc[max_profit_index+1:]
    max_profit_price = df_pair_timeframe.loc[max_profit_index, 'close']
    percentage_changes = [(price / max_profit_price - 1) * 100 for price in next_candels['close']]
    
    has_positive = any(change > 0 for change in percentage_changes)
    has_fifty_negative = any(change <= -50 for change in percentage_changes)
    has_five_negative = any(change <= -5 for change in percentage_changes)

    if (not has_positive and not has_fifty_negative and has_five_negative):
        print(f"Pair: {pair}")
        print(f"Timeframe: {timeframe}")


Pair: BNB/ETH
Timeframe: 1d
Pair: XVG/ETH
Timeframe: 1d
Pair: SC/ETH
Timeframe: 1d
